In [ ]:
from io import StringIO
import os
import boto3
import pandas as pd

In [ ]:
aws_access_key_id=os.getenv("AWS_ACCESS_KEY")
aws_secret_key=os.getenv("AWS_SECRET_KEY")

In [ ]:
aws_access_key_id

In [ ]:
def read_csv_from_s3(bucket: str,path: str, filename:str) ->pd.DataFrame:
    
    s3=boto3.client("s3",aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_key) 
    full_path = f"{path}{filename}"
    
    object = s3.get_object(Bucket=bucket, Key=full_path)
    object = object["Body"].read().decode("utf-8")
    output_df = pd.read_csv(StringIO(object))

    return output_df

In [ ]:
s3=boto3.client("s3",aws_access_key_id=aws_access_key_id,aws_secret_access_key=aws_secret_key) 

bucket="dataengineerea"


community_areas_path= "transformed_data/community_areas/"
company_path="transformed_data/company/"
date_path="transformed_data/date/"
payment_type_path="transformed_data/payment_type/"
taxi_trips_path="transformed_data/taxi_trips/"
weather_path="transformed_data/weather/"

In [ ]:
community_areas=payment_type = read_csv_from_s3(bucket=bucket,path=community_areas_path,filename="community_areas_master.csv")
company = read_csv_from_s3(bucket=bucket,path=company_path,filename="company_master.csv")
#date = read_csv_from_s3(bucket=bucket,path=date_path,filename="date_dimension.csv")
payment_type = read_csv_from_s3(bucket=bucket,path=paymnet_type_path,filename="payment_type_master.csv")

In [ ]:
trips_list=[]
weather_list=[]

In [ ]:
# Taxi dataa transformation and laoding
for file in s3.list_objects(Bucket=bucket, Prefix=taxi_trips_path)["Contents"]:
    taxi_trip_key = file['Key']
    
    if taxi_trip_key.split("/")[-1].strip() !="":
        if taxi_trip_key.split(".")[1] =="csv":
            filename=taxi_trip_key.split("/")[-1]
            trip=read_csv_from_s3(bucket, taxi_trips_path , filename)

            trips_list.append(trip)
            print(f"{filename} has been added")

In [ ]:
trips=pd.concat(trips_list, ignore_index=True)

In [ ]:
trips

In [ ]:
#Weather data transformation and loading
for file in s3.list_objects(Bucket=bucket, Prefix=weather_path)["Contents"]:
    weather_key = file["Key"]
    if weather_key.endswith(".csv"):
        print(f"Processing file: {weather_key}")
            
        filename=weather_key.split("/")[-1]
        weather_daily=read_csv_from_s3(bucket, weather_path , filename)
        weather_list.append(weather_daily)
        print(f"{filename} has been added")

In [ ]:
weather=pd.concat(weather_list, ignore_index=True)
weather

In [ ]:
trips_full=pd.merge(trips,weather,left_on="datetime_for_weather", right_on="datetime", how="inner")
trips_full=trips_full.drop(columns=["datetime"])
trips_full.head()